In [1]:
# Loading Libraries
import os
import numpy as np
from surprise import BaselineOnly
from surprise.dataset import Dataset
from surprise.reader import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import csv
import matplotlib.mlab as mlab
from surprise.model_selection import train_test_split

# Part 6: Naive collaborative ﬁltering